In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape


#import pandas as pd
target_X= pd.read_csv("../data/tem_train_input_7.csv").iloc[:,1:].values.astype(np.float32)
target_y =pd.read_csv("../data/tem_train_output_7.csv").iloc[:,1:].values.astype(np.float32)
test_X= pd.read_csv("../data/tem_val_input_7.csv").iloc[:,1:].values.astype(np.float32)
test_y =pd.read_csv("../data/tem_val_output_7.csv").iloc[:,1:].values.astype(np.float32)

X_train=target_X.astype(np.float32)
y_train=target_y.astype(np.float32)
#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

2024-10-09 03:36:33.713380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 03:36:33.785986: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-09 03:36:33.786008: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-09 03:36:34.140209: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],y_train.shape[1]

#################################################################################
# nbeats + I모델 생성 함수
def build_FC_model(input_length, output_length,dropout_rate=0.2,lossfn='mse' ):
    # 입력 레이어 정의
    input_seq = layers.Input(shape=(input_length, 1))

    # LSTM 레이어 추가
    lstm_output = layers.Dense(128,activation='linear')(input_seq)
    lstm_output = layers.Dropout(dropout_rate)(lstm_output)

    # 밀집층 추가
    dense_output = layers.Dense(64, activation='linear')(lstm_output)
    dense_output = layers.Dropout(dropout_rate)(dense_output)
    
    # 출력 크기에 맞춰서 리쉐이프
    dense_output = layers.Reshape((output_length, -1))(dense_output)

    # 출력 레이어
    output = layers.Dense(1, activation='linear')(dense_output)

    # 모델 정의
    model = models.Model(inputs=input_seq, outputs=output)

    # 모델 컴파일
    model.compile(optimizer='adam', loss=lossfn)

    
    return model
###############################################################################
def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = build_FC_model(backcast,forecast )
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅
#################################################################################


##########################################################################################

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [3]:
model_num = 5

mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,0.001)
mape_models = train_bagging_models(model_num, 'mape',2000,10,8,0.001)
mae_models = train_bagging_models(model_num, 'mae',2000,10,8,0.001)
mse_models = train_bagging_models(model_num, 'mse',2000,10,8,0.001)

2024-10-09 03:36:35.009073: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-09 03:36:35.009116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-10-09 03:36:35.009675: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'#######################################

In [11]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2, test_X)
mape_predictions_G = bagging_predict2(pred3, test_X)
mae_predictions_G = bagging_predict2(pred4, test_X)
mse_predictions_G = bagging_predict2(pred5, test_X)

concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G,mae_predictions_G,mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
pd.DataFrame(fin_pred_G.flatten()).to_csv("../result7/vanila_FC.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

12/12 [==============================] - 0s 1ms/step


(5.671629, 4.4074736)

In [13]:
concat_G = np.concatenate([mae_predictions_G, mase_predictions_G,mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
pd.DataFrame(fin_pred_G.flatten()).to_csv("../best7/vanila_FC.csv")

#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(5.6690474, 4.400086)

In [5]:
concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(5.67204, 4.4069242)

In [6]:
concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(5.7228603, 4.4315963)

In [7]:
concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(18.092987, 15.481022)

In [8]:
concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(5.63949, 4.355749)

In [9]:
concat_G = np.concatenate([mae_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(5.672762, 4.4026556)

In [10]:
concat_G = np.concatenate([mse_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten())

(5.720337, 4.471063)